# ML DEFRA Data Preparation for Air Quality Prediction

This notebook prepares DEFRA data for machine learning models - mirrors LAQN ml_prep exactly.

## What this notebook does

1. Loads cleaned DEFRA data from the optimised folder.

   ```bash
   ├── optimised/
   │   ├── 2023measurements/
   │   │   ├── Station_Name/
   │   │   │   ├── NO2__2023_01.csv
   │   │   │   ├── PM10__2023_01.csv
   │   │   │   └── ...
   │   ├── 2024measurements/
   │   └── 2025measurements/
   ```

2. Combines all measurements into a single dataset.
3. Creates temporal features (hour, day, month).
4. Creates sequences for ML training.

## DEFRA CSV Structure

Each CSV file contains:
```
timestamp, value, timeseries_id, station_name, pollutant_name, pollutant_std, latitude, longitude
```

## Output path:

Data will be saved to: `data/defra/ml_prep/`

In [1]:
# Standard imports same as LAQN
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Save section
import joblib

# Visualisation
import matplotlib.pyplot as plt

# Preprocessing libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## File Paths

- Usual drill, adding paths under this cell for organisation.

In [16]:
# DEFRA prep file paths
base_dir = Path.cwd().parent.parent / "data" / "defra"
project_root = Path.cwd() / "defra_ml_prep.ipynb"

# DEFRA optimised data path
optimised_path = base_dir / "optimised"

# Output paths
output_path = base_dir / "ml_prep"
output_path.mkdir(parents=True, exist_ok=True)

# Visualisation output path
visualisation_path = output_path / "visualisation"
visualisation_path.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {base_dir}")
print(f"Data path: {optimised_path}")
print(f"Output path: {output_path}")
print(f"Path exists: {optimised_path.exists()}")

Base directory: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra
Data path: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra/optimised
Output path: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra/ml_prep
Path exists: True


## 1) Load DEFRA data

DEFRA structure is nested: `optimised/YEARmeasurements/Station_Name/Pollutant__YYYY_MM.csv`

CSV columns: `timestamp, value, timeseries_id, station_name, pollutant_name, pollutant_std, latitude, longitude`

We'll map these to match LAQN column names for consistency:
- `timestamp` → `@MeasurementDateGMT`
- `value` → `@Value`
- `pollutant_std` → `SpeciesCode`
- `station_name` → `SiteCode` (with spaces replaced)

In [17]:
def load_defra_data(optimised_path):
    """
    Function to load the optimased data files from the defra dataset.

            optimised_path: path for data/defra/optimased directory.

    """
    optimised_path = Path(optimised_path)
    all_files = []
    file_count = 0
    
    # Get year measurement folders (2023measurements, 2024measurements, etc.)
    year_folders = sorted([f for f in optimised_path.iterdir() 
                          if f.is_dir() and 'measurements' in f.name])
    
    print(f"Found {len(year_folders)} year folders")
    
    for year_dir in year_folders:
        year_file_count = 0
        
        # Get station folders inside each year
        station_folders = sorted([f for f in year_dir.iterdir() if f.is_dir()])
        
        for station_dir in station_folders:
            # Get all CSV files in station folder
            for csv_file in station_dir.glob("*.csv"):
                try:
                    df = pd.read_csv(csv_file)
                    all_files.append(df)
                    file_count += 1
                    year_file_count += 1
                except Exception as e:
                    print(f"Error reading {csv_file.name}: {e}")
        
        print(f"  Loaded {year_dir.name}: {year_file_count} files")
    
    # Combine all dataframes
    if not all_files:
        raise ValueError(f"No CSV files found in {optimised_path}")
    
    combined_df = pd.concat(all_files, ignore_index=True)
    
    print(f"\n" + "="*40)
    print(f"Total files loaded: {file_count}")
    print(f"Total rows: {len(combined_df):,}")
    print(f"Columns: {list(combined_df.columns)}")
    
    return combined_df

In [18]:
# Load all defra data
df_raw = load_defra_data(optimised_path)
df_raw.head()

Found 3 year folders
  Loaded 2023measurements: 1431 files
  Loaded 2024measurements: 1193 files
  Loaded 2025measurements: 939 files

Total files loaded: 3563
Total rows: 2,525,991
Columns: ['timestamp', 'value', 'timeseries_id', 'station_name', 'pollutant_name', 'pollutant_std', 'latitude', 'longitude']


,timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std,latitude,longitude
0,2023-09-01 00:00:00,26.966,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
1,2023-09-01 01:00:00,27.349,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
2,2023-09-01 02:00:00,22.567,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
3,2023-09-01 04:00:00,17.021,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
4,2023-09-01 05:00:00,23.141,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055


## 2) Rename columns to match LAQN format

This ensures the rest of the pipeline (from LAQN) works without changes.

| DEFRA Column | LAQN Column | Description |
|--------------|-------------|-------------|
| timestamp | @MeasurementDateGMT | Datetime |
| value | @Value | Measurement |
| pollutant_std | SpeciesCode | Pollutant code |
| station_name | SiteName | Station name |
| (derived) | SiteCode | Station code (no spaces) |

In [19]:
# Rename columns to match LAQN format
df_raw = df_raw.rename(columns={
    'timestamp': '@MeasurementDateGMT',
    'value': '@Value',
    'pollutant_std': 'SpeciesCode',
    'station_name': 'SiteName'
})

# Create SiteCode from SiteName (replace spaces with underscores)
df_raw['SiteCode'] = df_raw['SiteName'].str.replace(' ', '_')

print(f"Renamed columns: {df_raw.columns.tolist()}")
df_raw.head()

Renamed columns: ['@MeasurementDateGMT', '@Value', 'timeseries_id', 'SiteName', 'pollutant_name', 'SpeciesCode', 'latitude', 'longitude', 'SiteCode']


,@MeasurementDateGMT,@Value,timeseries_id,SiteName,pollutant_name,SpeciesCode,latitude,longitude,SiteCode
0,2023-09-01 00:00:00,26.966,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055,Borehamwood_Meadow_Park
1,2023-09-01 01:00:00,27.349,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055,Borehamwood_Meadow_Park
2,2023-09-01 02:00:00,22.567,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055,Borehamwood_Meadow_Park
3,2023-09-01 04:00:00,17.021,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055,Borehamwood_Meadow_Park
4,2023-09-01 05:00:00,23.141,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055,Borehamwood_Meadow_Park


Renamed columns: ['@MeasurementDateGMT', '@Value', 'timeseries_id', 'SiteName', 'pollutant_name', 'SpeciesCode', 'latitude', 'longitude', 'SiteCode']

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>@MeasurementDateGMT</th>
      <th>@Value</th>
      <th>timeseries_id</th>
      <th>SiteName</th>
      <th>pollutant_name</th>
      <th>SpeciesCode</th>
      <th>latitude</th>
      <th>longitude</th>
      <th>SiteCode</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2023-09-01 00:00:00</td>
      <td>26.966</td>
      <td>4566</td>
      <td>Borehamwood Meadow Park</td>
      <td>Nitrogen oxides</td>
      <td>NOx</td>
      <td>51.661229</td>
      <td>-0.27055</td>
      <td>Borehamwood_Meadow_Park</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2023-09-01 01:00:00</td>
      <td>27.349</td>
      <td>4566</td>
      <td>Borehamwood Meadow Park</td>
      <td>Nitrogen oxides</td>
      <td>NOx</td>
      <td>51.661229</td>
      <td>-0.27055</td>
      <td>Borehamwood_Meadow_Park</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2023-09-01 02:00:00</td>
      <td>22.567</td>
      <td>4566</td>
      <td>Borehamwood Meadow Park</td>
      <td>Nitrogen oxides</td>
      <td>NOx</td>
      <td>51.661229</td>
      <td>-0.27055</td>
      <td>Borehamwood_Meadow_Park</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2023-09-01 04:00:00</td>
      <td>17.021</td>
      <td>4566</td>
      <td>Borehamwood Meadow Park</td>
      <td>Nitrogen oxides</td>
      <td>NOx</td>
      <td>51.661229</td>
      <td>-0.27055</td>
      <td>Borehamwood_Meadow_Park</td>
    </tr>
    <tr>
      <th>4</th>
      <td>2023-09-01 05:00:00</td>
      <td>23.141</td>
      <td>4566</td>
      <td>Borehamwood Meadow Park</td>
      <td>Nitrogen oxides</td>
      <td>NOx</td>
      <td>51.661229</td>
      <td>-0.27055</td>
      <td>Borehamwood_Meadow_Park</td>
    </tr>
  </tbody>
</table>
</div>


## 3) Handle DEFRA Missing Value Flags

From your DEFRA report:
- DEFRA uses `-99` for maintenance/calibration
- DEFRA uses `-1` for invalid data
- Also has some negative values near detection limits (631 total, 0.025%)

Replace these with NaN for consistent handling.

In [20]:
def handle_flags(df, value_col='@Value'):
    """
    Replace defra quality flags with NaN.
    
    -99: Station maintenance or calibration
    -1: Other invalid data or insufficient capture
    Negative values: Near detection limit (replace with nan)
    """
    df = df.copy()
    
    # Count flags before replacement
    flag_99 = (df[value_col] == -99).sum()
    flag_1 = (df[value_col] == -1).sum()
    negative = (df[value_col] < 0).sum()
    
    print(f"defra quality flags found:")
    print(f"  -99 (maintenance): {flag_99:,} ({flag_99/len(df)*100:.2f}%)")
    print(f"  -1 (invalid): {flag_1:,} ({flag_1/len(df)*100:.2f}%)")
    print(f"  All negative values: {negative:,} ({negative/len(df)*100:.2f}%)")
    
    # Replace -99 and -1 flags with nan
    df[value_col] = df[value_col].replace([-99, -1], np.nan)
    
    # Replace remaining negative values with NaN (near detection limit)
    df.loc[df[value_col] < 0, value_col] = np.nan
    
    remaining_neg = (df[value_col] < 0).sum()
    print(f"\nAfter cleaning: {remaining_neg} negative values remain.")
    
    return df

# Apply flag handling
df_raw = handle_flags(df_raw)

defra quality flags found:
  -99 (maintenance): 0 (0.00%)
  -1 (invalid): 0 (0.00%)
  All negative values: 0 (0.00%)

After cleaning: 0 negative values remain.


defra quality flags found:
  -99 (maintenance): 0 (0.00%)
  -1 (invalid): 0 (0.00%)
  All negative values: 0 (0.00%)

After cleaning: 0 negative values remain.

## 4) Data exploration

Already checked data many times but I think it is beneficial to add it here again:

- How many unique stations? - **18**
- Which pollutants (species)? - **37 pollutants** (6 regulatory + 31 VOCs)
- Date range? - **01.01.2023 to 18.11.2025** (35 months)
- Total measurements? - **2,525,991 records**
- Data completeness? - **91.20%**

### Comparison with LAQN:

| Metric | LAQN | DEFRA |
|--------|------|-------|
| Stations | 78 | 18 |
| Pollutants | 6 | 37 |
| Completeness | 87.13% | 91.20% |
| Total records | 3,446,208 | 2,525,991 |

In [21]:
# Define colm names based on optimised data structure
date_col = '@MeasurementDateGMT'
value_col = '@Value'
site_col = 'SiteCode'
species_col = 'SpeciesCode'

# Convert datetime
df_raw[date_col] = pd.to_datetime(df_raw[date_col])

# run them.
print(f"Unique sites: {df_raw[site_col].nunique()}")
print(f"Unique species: {df_raw[species_col].nunique()}")
print(f"\nDate range: {df_raw[date_col].min()} to {df_raw[date_col].max()}")
print(f"\nSpecies in data:")
print(df_raw[species_col].value_counts())

Unique sites: 18
Unique species: 37

Date range: 2023-01-01 01:00:00 to 2025-11-09 23:00:00

Species in data:
SpeciesCode
NO2                326072
NO                 326061
NOx                325387
PM2.5              234748
PM10               227142
O3                 194333
SO2                 72928
CO                  48578
1,3,5-TMB           26649
i-Octane            26649
Benzene             26649
1,2,3-TMB           26649
Ethylbenzene        26649
Toluene             26649
o-Xylene            26649
n-Octane            26649
n-Heptane           26649
1,2,4-TMB           26649
Propane             26618
n-Pentane           26618
i-Pentane           26618
Ethene              26618
Propene             26618
Isoprene            26618
i-Hexane            26599
trans-2-Butene      26599
trans-2-Pentene     26599
n-Butane            26599
1-Butene            26599
i-Butane            26599
Ethane              26599
cis-2-Butene        26599
n-Hexane            26580
1-Pentene           

    Unique sites: 18
    Unique species: 37

    Date range: 2023-01-01 01:00:00 to 2025-11-09 23:00:00

    Species in data:
    SpeciesCode
    NO2                326072
    NO                 326061
    NOx                325387
    PM2.5              234748
    PM10               227142
    O3                 194333
    SO2                 72928
    CO                  48578
    1,3,5-TMB           26649
    i-Octane            26649
    Benzene             26649
    1,2,3-TMB           26649
    Ethylbenzene        26649
    Toluene             26649
    o-Xylene            26649
    n-Octane            26649
    n-Heptane           26649
    1,2,4-TMB           26649
    Propane             26618
    n-Pentane           26618
    i-Pentane           26618
    Ethene              26618
    Propene             26618
    Isoprene            26618
    i-Hexane            26599
    trans-2-Butene      26599
    trans-2-Pentene     26599
    n-Butane            26599
    1-Butene            26599
    i-Butane            26599
    Ethane              26599
    cis-2-Butene        26599
    n-Hexane            26580
    1-Pentene           26572
    1,3-Butadiene       26568
    Ethyne              26529
    m,p-Xylene          25503
    Name: count, dtype: int64

## 5) Selecting target pollutants

As documented in `/docs/DEFRA_data_quality.md` and `/docs/LAQN_DEFRA_benchmark.md`, DEFRA's highest coverage pollutants are:

| Pollutant | Stations | Coverage |
|-----------|----------|----------|
| PM10      | 15       | 10.42%   |
| PM2.5     | 15       | 10.42%   |
| NO2       | 14       | 9.72%    |
| NOx       | 14       | 9.72%    |
| NO        | 14       | 9.72%    |
| O3        | 9        | 6.25%    |

### Comparison with LAQN coverage:

| Pollutant | LAQN Sites | DEFRA Sites |
|-----------|------------|-------------|
| NO2       | 60         | 14          |
| PM2.5     | 53         | 15          |
| PM10      | 43         | 15          |
| O3        | 11         | 9           |

### Selected for ML training:

To enable direct comparison with LAQN results, filtering to:
- **NO2** (14 stations) - primary traffic pollutant
- **PM10** (15 stations) - particulate matter
- **O3** (9 stations) - secondary photochemical pollutant

**Note:** DEFRA has fewer stations (18 total vs LAQN's 78) but higher data completeness (91.2% vs 87.1%). This tests whether data quality compensates for spatial coverage.

In [22]:
# target pollutants
target_pollutants = ['NO2', 'PM25', 'PM10', 'O3']

# Filter data
df_filtered = df_raw[df_raw[species_col].isin(target_pollutants)].copy()

print(f"Rows before filtering: {len(df_raw):,}")
print(f"Rows after filtering: {len(df_filtered):,}")
print(f"\nPollutants included:")
print(df_filtered[species_col].value_counts())

Rows before filtering: 2,525,991
Rows after filtering: 747,547

Pollutants included:
SpeciesCode
NO2     326072
PM10    227142
O3      194333
Name: count, dtype: int64


    Rows before filtering: 2,525,991
    Rows after filtering: 747,547

    Pollutants included:
    SpeciesCode
    NO2     326072
    PM10    227142
    O3      194333
    Name: count, dtype: int64

### 5) Selecting Target Pollutant Results:

Comparison of LAQN and DEFRA metrics:

### Filtering results:

| Metric | Before | After |
|--------|--------|-------|
| Total rows | 2,525,991 | 747,547 |
| Pollutants | 37 | 3 |
| Data retained | 100% | 29.6% |

### Selected pollutants breakdown:

| Pollutant | Records | Percentage | Stations |
|-----------|---------|------------|----------|
| NO2 | 326,072 | 43.6% | 14 |
| PM10 | 227,142 | 30.4% | 15 |
| O3 | 194,333 | 26.0% | 9 |

### Why these three?

1. **Direct comparison with LAQN** - same pollutants used in LAQN small sample training
2. **Regulatory relevance** - NO2, PM10, O3 are key UK Air Quality Standards pollutants
3. **Sufficient coverage** - 9-15 stations per pollutant provides spatial diversity

### Comparison with LAQN filtered data:

| Aspect | LAQN | DEFRA |
|--------|------|-------|
| Rows after filter | ~3.4M | 747,547 |
| NO2 stations | 60 | 14 |
| PM10 stations | 43 | 15 |
| O3 stations | 11 | 9 |

**Note:** DEFRA has ~4.5x fewer records after filtering, but higher per-station completeness. This tests whether quality > quantity for ML performance.

## 6) Temporal feature adding

Already added this feature in analysis part.

Temporal features help the model understand pollution patterns:

- **Hour of day** - captures rush hour peaks (morning 7-9am, evening 5-7pm)
- **Day of week** - weekday vs weekend traffic differences
- **Month/Season** - seasonal variations (winter inversions, summer ozone)
- **Is weekend** - binary flag for traffic pattern changes

These features help the model learn when pollution is typically high or low.

### DEFRA seasonal patterns (from seasonal_averages.csv):

| Pollutant | Winter | Spring | Summer | Autumn | Pattern |
|-----------|--------|--------|--------|--------|---------|
| NO2 (µg/m³) | 25.43 | 20.42 | 15.51 | 22.86 | Winter peak |
| PM10 (µg/m³) | 14.14 | 15.03 | 11.34 | 12.45 | Spring peak |
| PM2.5 (µg/m³) | 9.00 | 8.99 | 5.92 | 7.38 | Winter peak |
| O3 (µg/m³) | 38.78 | 55.40 | 51.25 | 38.60 | Spring/Summer peak |

**Key observations:**
- **NO2** highest in winter (25.43), lowest in summer (15.51) - heating emissions + reduced dispersion
- **PM10** highest in spring (15.03), lowest in summer (11.34) - combustion + stagnant air
- **PM2.5** highest in winter (9.00), lowest in summer (5.92) - domestic burning + temperature inversions
- **O3** highest in spring (55.40), lowest in autumn (38.60) - photochemical formation from sunlight

**Note:** DEFRA's 91.2% completeness means fewer gaps in temporal sequences compared to LAQN, potentially improving the model's ability to learn these patterns.

In [23]:
def temporal_features(df, datetime_col):
    """
     Temporal features from datetime column.
    Param:
            df : pandas.DataFrame

            datetime_col : str
       
    """
    df = df.copy()
    
    #  datetime type needs to be ensured
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    
    # Extract features
    df['hour'] = df[datetime_col].dt.hour
    df['day_of_week'] = df[datetime_col].dt.dayofweek  # 0=Monday, 6=Sunday
    df['month'] = df[datetime_col].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    print("Added temporal features: hour, day_of_week, month, is_weekend")
    
    return df

In [24]:
# Add temporal features
df_temporal = temporal_features(df_filtered, date_col)

# Preview
df_temporal[['hour', 'day_of_week', 'month', 'is_weekend']].head()

Added temporal features: hour, day_of_week, month, is_weekend


,hour,day_of_week,month,is_weekend
2918,1,6,1,1
2919,2,6,1,1
2920,3,6,1,1
2921,4,6,1,1
2922,5,6,1,1


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>hour</th>
      <th>day_of_week</th>
      <th>month</th>
      <th>is_weekend</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2918</th>
      <td>1</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2919</th>
      <td>2</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2920</th>
      <td>3</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2921</th>
      <td>4</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2922</th>
      <td>5</td>
      <td>6</td>
      <td>1</td>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

Added temporal features: hour, day_of_week, month, is_weekend


## 7) Wide formatting

For ml (Air quality prediction using CNN+LSTM-based hybrid deep learning architecture', *Environmental Science and Pollution Research*, 29(8), pp. 11920-11938)

And according to their search here are the findings:
 | Method      | Input                                    | Output           |
| ----------- | ---------------------------------------- | ---------------- |
| UNI/UNI     | Historical info of target pollutant only | Single pollutant |
| MULTI/UNI   | Historical info of all pollutants        | Single pollutant |
| MULTI/MULTI | Historical info of all pollutants        | All pollutants   |

page 11922 (Results section):

> "The multivariate model without using meteorological data revealed the best results."

So order to create multivariate input I will be formatting pivot to wider,adding each station/species combination to table.




In [25]:
def wide_format(df, datetime_col, site_col, species_col, value_col):
    """
    Pivot data from long to wide format. 
    Each site-species combination becomes a column.
    Each row represents one timestamp.
    
    Params:
        datetime_col, site_col, species_col, value_col 
    """
    df = df.copy()
    
    # Create site_species identifier
    df['site_species'] = df[site_col] + '_' + df[species_col]
    
    # Pivot table
    # If duplicate datetime-site_species combinations exist, take mean
    pivoted = df.pivot_table(
        index=datetime_col,
        columns='site_species',
        values=value_col,
        aggfunc='mean'
    )
    
    # Sort by datetime
    pivoted = pivoted.sort_index()
    
    print(f"Created wide format:")
    print(f"Timestamps: {len(pivoted):,}")
    print(f"Features (site-species): {len(pivoted.columns)}")
    print(f"Date range: {pivoted.index.min()} to {pivoted.index.max()}")
    
    return pivoted

In [26]:
# Create wide format
df_wide = wide_format(df_temporal, date_col, site_col, species_col, value_col)

# Preview
print("\nFirst 10 columns:")
print(list(df_wide.columns)[:10])
df_wide.head()

Created wide format:
Timestamps: 24,355
Features (site-species): 37
Date range: 2023-01-01 01:00:00 to 2025-11-09 23:00:00

First 10 columns:
['Borehamwood_Meadow_Park_NO2', 'Borehamwood_Meadow_Park_PM10', 'Camden_Kerbside_NO2', 'Camden_Kerbside_PM10', 'Ealing_Horn_Lane_PM10', 'Haringey_Roadside_NO2', 'London_Bexley_NO2', 'London_Bexley_PM10', 'London_Bloomsbury_NO2', 'London_Bloomsbury_O3']


site_species,Borehamwood_Meadow_Park_NO2,Borehamwood_Meadow_Park_PM10,Camden_Kerbside_NO2,Camden_Kerbside_PM10,Ealing_Horn_Lane_PM10,Haringey_Roadside_NO2,London_Bexley_NO2,London_Bexley_PM10,London_Bloomsbury_NO2,London_Bloomsbury_O3,...,London_N._Kensington_NO2,London_N._Kensington_O3,London_N._Kensington_PM10,London_Norbury_Manor_School_PM10,London_Teddington_Bushy_Park_PM10,London_Westminster_NO2,London_Westminster_O3,Southwark_A2_Old_Kent_Road_NO2,Southwark_A2_Old_Kent_Road_PM10,Tower_Hamlets_Roadside_NO2
@MeasurementDateGMT,,,,,,,,,,,,,,,,,,,,,
2023-01-01 01:00:00,4.781,8.5,7.268,22.223,NaN,12.240,1.530,NaN,5.293,67.721,...,2.104,65.259,10.7,NaN,8.550,8.396,NaN,5.546,22.223,5.355
2023-01-01 02:00:00,5.164,8.0,9.563,8.696,NaN,8.033,2.678,NaN,4.587,70.448,...,1.721,66.457,8.7,NaN,11.300,7.254,NaN,4.973,10.628,4.781
2023-01-01 03:00:00,2.678,8.8,5.546,14.493,NaN,6.311,1.339,NaN,3.732,70.598,...,0.956,68.053,10.9,NaN,13.350,5.865,NaN,4.399,12.561,6.311
2023-01-01 04:00:00,2.486,11.3,4.973,17.392,NaN,5.737,1.339,NaN,2.963,72.893,...,0.574,69.650,12.9,NaN,15.725,4.864,NaN,3.825,16.425,3.825
2023-01-01 05:00:00,2.869,11.4,5.164,16.425,NaN,7.459,1.530,NaN,3.300,70.847,...,1.147,67.854,14.8,NaN,17.550,4.195,NaN,3.634,20.290,3.442


Created wide format:
Timestamps: 24,355
Features (site-species): 37
Date range: 2023-01-01 01:00:00 to 2025-11-09 23:00:00

First 10 columns:
['Borehamwood_Meadow_Park_NO2', 'Borehamwood_Meadow_Park_PM10', 'Camden_Kerbside_NO2', 'Camden_Kerbside_PM10', 'Ealing_Horn_Lane_PM10', 'Haringey_Roadside_NO2', 'London_Bexley_NO2', 'London_Bexley_PM10', 'London_Bloomsbury_NO2', 'London_Bloomsbury_O3']

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>site_species</th>
      <th>Borehamwood_Meadow_Park_NO2</th>
      <th>Borehamwood_Meadow_Park_PM10</th>
      <th>Camden_Kerbside_NO2</th>
      <th>Camden_Kerbside_PM10</th>
      <th>Ealing_Horn_Lane_PM10</th>
      <th>Haringey_Roadside_NO2</th>
      <th>London_Bexley_NO2</th>
      <th>London_Bexley_PM10</th>
      <th>London_Bloomsbury_NO2</th>
      <th>London_Bloomsbury_O3</th>
      <th>...</th>
      <th>London_N._Kensington_NO2</th>
      <th>London_N._Kensington_O3</th>
      <th>London_N._Kensington_PM10</th>
      <th>London_Norbury_Manor_School_PM10</th>
      <th>London_Teddington_Bushy_Park_PM10</th>
      <th>London_Westminster_NO2</th>
      <th>London_Westminster_O3</th>
      <th>Southwark_A2_Old_Kent_Road_NO2</th>
      <th>Southwark_A2_Old_Kent_Road_PM10</th>
      <th>Tower_Hamlets_Roadside_NO2</th>
    </tr>
    <tr>
      <th>@MeasurementDateGMT</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2023-01-01 01:00:00</th>
      <td>4.781</td>
      <td>8.5</td>
      <td>7.268</td>
      <td>22.223</td>
      <td>NaN</td>
      <td>12.240</td>
      <td>1.530</td>
      <td>NaN</td>
      <td>5.293</td>
      <td>67.721</td>
      <td>...</td>
      <td>2.104</td>
      <td>65.259</td>
      <td>10.7</td>
      <td>NaN</td>
      <td>8.550</td>
      <td>8.396</td>
      <td>NaN</td>
      <td>5.546</td>
      <td>22.223</td>
      <td>5.355</td>
    </tr>
    <tr>
      <th>2023-01-01 02:00:00</th>
      <td>5.164</td>
      <td>8.0</td>
      <td>9.563</td>
      <td>8.696</td>
      <td>NaN</td>
      <td>8.033</td>
      <td>2.678</td>
      <td>NaN</td>
      <td>4.587</td>
      <td>70.448</td>
      <td>...</td>
      <td>1.721</td>
      <td>66.457</td>
      <td>8.7</td>
      <td>NaN</td>
      <td>11.300</td>
      <td>7.254</td>
      <td>NaN</td>
      <td>4.973</td>
      <td>10.628</td>
      <td>4.781</td>
    </tr>
    <tr>
      <th>2023-01-01 03:00:00</th>
      <td>2.678</td>
      <td>8.8</td>
      <td>5.546</td>
      <td>14.493</td>
      <td>NaN</td>
      <td>6.311</td>
      <td>1.339</td>
      <td>NaN</td>
      <td>3.732</td>
      <td>70.598</td>
      <td>...</td>
      <td>0.956</td>
      <td>68.053</td>
      <td>10.9</td>
      <td>NaN</td>
      <td>13.350</td>
      <td>5.865</td>
      <td>NaN</td>
      <td>4.399</td>
      <td>12.561</td>
      <td>6.311</td>
    </tr>
    <tr>
      <th>2023-01-01 04:00:00</th>
      <td>2.486</td>
      <td>11.3</td>
      <td>4.973</td>
      <td>17.392</td>
      <td>NaN</td>
      <td>5.737</td>
      <td>1.339</td>
      <td>NaN</td>
      <td>2.963</td>
      <td>72.893</td>
      <td>...</td>
      <td>0.574</td>
      <td>69.650</td>
      <td>12.9</td>
      <td>NaN</td>
      <td>15.725</td>
      <td>4.864</td>
      <td>NaN</td>
      <td>3.825</td>
      <td>16.425</td>
      <td>3.825</td>
    </tr>
    <tr>
      <th>2023-01-01 05:00:00</th>
      <td>2.869</td>
      <td>11.4</td>
      <td>5.164</td>
      <td>16.425</td>
      <td>NaN</td>
      <td>7.459</td>
      <td>1.530</td>
      <td>NaN</td>
      <td>3.300</td>
      <td>70.847</td>
      <td>...</td>
      <td>1.147</td>
      <td>67.854</td>
      <td>14.8</td>
      <td>NaN</td>
      <td>17.550</td>
      <td>4.195</td>
      <td>NaN</td>
      <td>3.634</td>
      <td>20.290</td>
      <td>3.442</td>
    </tr>
  </tbody>
</table>
<p>5 rows × 37 columns</p>
</div>

## 7) NaN value handling

DEFRA dataset has 91.2% completeness (vs LAQN's 87.1%), but still need to handle missing values.

### Strategy:

1. **Remove high-missing columns** - drop site-species combinations with >30% missing
2. **Interpolate small gaps** - linear interpolation for gaps up to 5 hours
3. **Drop remaining NaN rows** - remove rows that still have missing values

### DEFRA-specific considerations:

From the data quality report, missing values are unevenly distributed:

| Station | Issue | Missing % |
|---------|-------|-----------|
| London Eltham | PM10 equipment failure | 97.09% |
| London Teddington | PM sensor maintenance | ~43% |
| Other stations | Normal gaps | <5% |

These problematic station-pollutant combinations will be automatically removed in step 1 (>30% missing threshold).

### Why this approach?

- **Interpolation limit of 5 hours** - pollution patterns are autocorrelated, so short gaps can be reasonably estimated
- **30% threshold** - keeps stations with genuine data while removing equipment failures
- **No imputation for long gaps** - avoids introducing artificial patterns that could mislead the model
